In [1]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2



from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model


from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


from imutils import paths
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
from zipfile import ZipFile
file_name = "my_data.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [3]:
INIT_LR=1e-4
EPOCHS=30
BS=32
DIRECTORY="my_data/"

In [4]:
#from PIL import ImageFile, Image
#ImageFile.LOAD_TRUNCATED_IMAGES = False

In [5]:
data=[]
labels=[]

for category in os.listdir(DIRECTORY):
    path= os.path.join(DIRECTORY,category)
    for img in os.listdir(path):
        img_path =os.path.join(path,img)
        image=load_img(img_path,target_size=(224,224))
        image=img_to_array(image)
        image=preprocess_input(image)
        
        data.append(image)
        labels.append(category)
print(labels[:10])       


['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat']


In [6]:
#perform one-hot encoding

lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

data=np.array(data,dtype="float32")
labels = np.array(labels)

In [7]:
(trainX,testX,trainY,testY)=train_test_split(data,labels,test_size=0.20,stratify=labels,random_state=42)

In [8]:
# construct the training image generator for data augementation

aug=ImageDataGenerator(rotation_range=20,
                       zoom_range=0.15,
                       width_shift_range=0.2,
                       shear_range=0.15,
                       horizontal_flip=True,
                       fill_mode="nearest")

In [9]:
#load the model

baseModel=MobileNetV2(weights="imagenet",include_top=False,
                     input_tensor=Input(shape=(224,224,3)))

In [10]:
#construct the head of the model that will be placed on the top of the best model
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name="flatten")(headModel)
headModel=Dense(128,activation="relu")(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation="softmax")(headModel)

In [11]:
#place the head FC model on the top of the base model
model = Model(inputs=baseModel.input,outputs=headModel)

In [12]:
#loop over all layers in the base model

for layer in baseModel.layers:
    layer.trainable =False

In [13]:
#complie our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR,decay=INIT_LR/EPOCHS)
model.compile(loss="binary_crossentropy",optimizer=opt,
             metrics=['accuracy'])

[INFO] compiling model...


F:\anaconda\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:113: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [14]:
# trsin the heard of network

print("[INFO] training head...")

h=model.fit(aug.flow(trainX,trainY,batch_size=BS),
            steps_per_epoch=len(trainX)//BS,
            validation_data=(testX,testY),
            validation_steps=len(testX)//BS,
            epochs=EPOCHS)

[INFO] training head...
Epoch 1/30
52/52 [==============================] - 44s 759ms/step - loss: 0.4991 - accuracy: 0.8072 - val_loss: 0.2358 - val_accuracy: 0.9595
Epoch 2/30
52/52 [==============================] - 38s 725ms/step - loss: 0.2123 - accuracy: 0.9580 - val_loss: 0.1422 - val_accuracy: 0.9619
Epoch 3/30
52/52 [==============================] - 35s 668ms/step - loss: 0.1469 - accuracy: 0.9641 - val_loss: 0.0985 - val_accuracy: 0.9714
Epoch 4/30
52/52 [==============================] - 35s 677ms/step - loss: 0.1247 - accuracy: 0.9672 - val_loss: 0.0857 - val_accuracy: 0.9690
Epoch 5/30
52/52 [==============================] - 34s 656ms/step - loss: 0.1068 - accuracy: 0.9647 - val_loss: 0.0707 - val_accuracy: 0.9786
Epoch 6/30
52/52 [==============================] - 35s 667ms/step - loss: 0.0906 - accuracy: 0.9793 - val_loss: 0.0679 - val_accuracy: 0.9762
Epoch 7/30
52/52 [==============================] - 35s 680ms/step - loss: 0.0787 - accuracy: 0.9787 - val_loss: 0.072